## 2 - Introductory Example - 1-Axis tracker with cumulative sky

Method <b> Gencumsky </b> has been modified to divide the yearly-cumulative sky into various skies, each one representing the cumulative irradiance for the hours at which the tracker is at a certain angle. For faster running, for a tracker that moves between 45 and -45 degrees limit angle, if only positions every 5 degrees are considered (45, 40, 35 .... -4-, -45), then only 18 skies (and 18 simulations) will be run for the whole year.

![Example of the hemisphere cumulative sky](../images_wiki/Journal2Pics/tracking_cumulativesky.png)


This procedure was presented in:

Ayala Pelaez S, Deline C, Greenberg P, Stein JS, Kostuk RK. Model and validation of single-axis tracking with bifacial PV. IEEE J Photovoltaics. 2019;9(3):715–21. https://ieeexplore.ieee.org/document/8644027 and https://www.nrel.gov/docs/fy19osti/72039.pdf (pre-print, conference version)


### Steps:
<ol>
    <li> <a href='#step1'> Create a folder for your simulation, and load bifacial_radiance </a></li> 
    <li> <a href='#step2'> Create a Radiance Object </a></li> 
    <li> <a href='#step3'> Set the Albedo </a></li> 
    <li> <a href='#step4'> Download Weather Files </a></li>    
    <ul> (VERY SIMILAR TO FIXED TILT EXAMPLE UNTIL HERE) </ul> 
    <li> <a href='#step5'> Set Tracking Angles </a></li> 
    <li> <a href='#step6'> Generate the Sky </a></li> 
    <li> <a href='#step7'> Define a Module type </a></li> 
    <li> <a href='#step8'> Create the scene </a></li> 
    <li> <a href='#step9'> Combine Ground, Sky and Scene Objects </a></li> 
    <li> <a href='#step10'> Analyze and get results </a></li> 
    <li> <a href='#step11'> Clean Results </a></li>   
   
</ol>

And finally:  <ul> <a href='#condensed'> Condensed instructions </a></ul>   

<a id='step1'></a>


## 1. Create a folder for your simulation, and load bifacial_radiance 

First let's set the folder where the simulation will be saved. By default, this is the TEMP folder in the bifacial_radiance distribution.

The lines below find the location of the folder relative to this Jupyter Journa. You can alternatively point to an empty directory (it will open a load GUI Visual Interface) or specify any other directory in your computer, for example:

#### testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal2'



In [1]:
import os
testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP


This will load bifacial_radiance and other libraries from python that will be useful for this Jupyter Journal:

In [2]:
from bifacial_radiance import *
import numpy as np

<a id='step2'></a>

## 2. Create a Radiance Object

In [3]:
# Create a RadianceObj 'object' named bifacial_example. no whitespace allowed
demo = RadianceObj('bifacial_tracking_example', path = testfolder)  

path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP


This will create all the folder structure of the bifacial_radiance Scene in the designated testfolder in your computer, and it should look like this:


![Folder Structure](../images_wiki/Journal1Pics/folderStructure.png)

<a id='step3'></a>

## 3. Set the Albedo

To see more options of ground materials available (located on ground.rad), run this function without any input. 

In [4]:
# Input albedo number or material name like 'concrete'.  
demo.setGround()  # This prints available materials.

Input albedo 0-1, or ground material names:['black', 'litesoil', 'concrete', 'white_EPDM', 'beigeroof', 'beigeroof_lite', 'beigeroof_heavy', 'asphalt', 'greyroof', 'Metal_Aluminum_Anodized', 'Color_I11', 'M_0010_Snow', 'Metal_Grey']


If a number between 0 and 1 is passed, it assumes it's an albedo value. For this example, we want a natural-ground albedo value, so we'll use 0.25

In [5]:
albedo = 0.25
demo.setGround(albedo)

<a id='step4'></a>

## 4. Download and Load Weather Files

There are various options provided in bifacial_radiance to load weatherfiles. getEPW is useful because you just set the latitude and longitude of the location and it donwloads the meteorologicla data for any location. 

In [6]:
# Pull in meteorological data using pyEPW for any global lat/lon
epwfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.

Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!


The downloaded EPW will be in the EPWs folder.

To load the data, use readWeatherFile. This reads EPWs, TMY meterological data, or even your own data as long as it follows TMY data format (With any time resoultion).

In [7]:
# Read in the weather data pulled in above. 
metdata = demo.readWeatherFile(weatherFile = epwfile) 

## TRACKING Workflow

<a id='step5'></a>

Until now, all the steps looked the same from Tutorial 1. The following section follows similar steps, but the functions are specific for working with single axis tracking.

## 5. Set Tracking Angles

This function will read the weather file, and based on the sun position it will calculate the angle the tracker should be at for each hour. It will create metdata files for each of the tracker angles considered.

In [8]:
limit_angle = 5 # tracker rotation limit angle. Setting it ridiculously small so this runs faster.
angledelta = 5 # sampling between the limit angles. 
backtrack = True
gcr = 0.33
cumulativesky = True # This is important for this example!
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)

C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:449: RuntimeWarning: invalid value encountered in arccos
  wc = np.degrees(np.arccos(temp))
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:473: RuntimeWarning: invalid value encountered in arccos
  aoi = np.degrees(np.arccos(np.abs(np.sum(sun_vec*panel_norm, axis=0))))
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:568: RuntimeWarning: invalid value encountered in remainder
  surface_azimuth = surface_azimuth % 360
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:572: RuntimeWarning: invalid value encountered in arccos
  surface_tilt = 90 - np.degrees(np.arccos(dotproduct))


Saving file EPWs\1axis_-5.0.csv, # points: 2215
Saving file EPWs\1axis_0.0.csv, # points: 81
Saving file EPWs\1axis_5.0.csv, # points: 2087


Setting backtrack to True is important in this step, so the trackers correct for self-shading when following the sun at high zenith angles. 

<a id='step6'></a>

## 6. Generate the Sky

This will create the skies for each sub-metdata file created by set1axis.


In [9]:
trackerdict = demo.genCumSky1axis(trackerdict = trackerdict)

message: There were 2110 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-5.0.rad
message: There were 13 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_0.0.rad
message: There were 2067 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_5.0.rad


This is how one of the cumulative sky .cal files associated with each .rad file generated look like: 

![Example of the gencumsky1axis](../images_wiki/Journal2Pics/gencumsky1axis_example_file_structure_and_contents.png)


Each of the values corresponds to the cumulative rradiance of one of those patches, for when the tracker is at that specific angle through the year.

<a id='step7'></a>

## 7. Define the Module type

Let's make a more interesting module in this example. Let's do 2-up configuration in portrait, with the modules rotating around a 10 centimeter round torque tube. Let's add a gap between the two modules in 2-UP of 10 centimeters, as well as gap between the torque tube and the modules of 5 centimeters. Along the row, the modules are separated only 2 centimeters for this example. The torquetube is painted Metal_Grey in this example (it's one of the materials available in Ground.rad, and it is 40% reflective).


In [10]:
x = 0.984  # meters
y = 1.7    # meters
moduletype = 'Custom Tracker Module'
torquetube = True
tubetype = 'round'
diameter = 0.1 # diameter of the torque tube
numpanels = 2
axisofrotationTorqueTube = True
zgap = 0.05
ygap = 0.10
xgap = 0.02
material = 'Metal_Grey'

demo.makeModule(name = moduletype, x = x, y = y, 
                torquetube = torquetube, tubetype = tubetype, material = material,
                diameter = diameter, xgap = xgap, ygap = ygap, zgap = zgap, 
                numpanels = numpanels, axisofrotationTorqueTube = axisofrotationTorqueTube)


Module Name: Custom_Tracker_Module
REWRITING pre-existing module file. 
Module Custom Tracker Module successfully created


{'x': 0.984,
 'y': 1.7,
 'scenex': 1.004,
 'sceney': 3.5,
 'scenez': 0.1,
 'numpanels': 2,
 'bifi': 1,
 'text': '! genbox black Custom_Tracker_Module 0.984 1.7 0.02 | xform -t -0.492 -1.75 0.1 -a 2 -t 0 1.8 0\r\n! genrev Metal_Grey tube1 t*1.004 0.05 32 | xform -ry 90 -t -0.502 0 0',
 'modulefile': 'objects\\Custom_Tracker_Module.rad',
 'offsetfromaxis': 0.1,
 'xgap': 0.02,
 'ygap': 0.1,
 'zgap': 0.05,
 'cellModule': None,
 'torquetube': {'bool': True,
  'diameter': 0.1,
  'tubetype': 'round',
  'material': 'Metal_Grey'}}

<a id='step8'></a>

## 8. Make the Scene

The scene Dictionary specifies the information of the scene. For tracking, different input parameters are expected in the dictionary, such as number of rows, number of modules per row, row azimuth, hub_height (distance between the axis of rotation of the modules and the ground). 

In [11]:
hub_height = 2.3
sceneDict = {'gcr': gcr,'hub_height':hub_height, 'nMods': 20, 'nRows': 7}  


To make the scene we have to create a Scene Object through the method makeScene1axis. This method will create a .rad file in the objects folder, with the parameters specified in sceneDict and the module created above.

In [12]:
trackerdict = demo.makeScene1axis(trackerdict = trackerdict, moduletype = moduletype, sceneDict = sceneDict) 


Making .rad files for cumulativesky 1-axis workflow
3 Radfiles created in /objects/


<a id='step9'></a>

## 9. Combine Ground, Sky and Scene Objects

makeOct1axis joins the sky.rad file, ground.rad file, and the geometry.rad files created in makeScene.

In [13]:
trackerdict = demo.makeOct1axis(trackerdict = trackerdict)


Making 3 octfiles in root directory.
Created 1axis_-5.0.oct
Created 1axis_0.0.oct
Created 1axis_5.0.oct


<a id='step10'></a>

## 10. Analyze and get results

We can choose to analyze any module in the Scene we have created. The default, if no modWanted or rowWanted is passed, is to sample the center module of the center row. 

For this example we will sample row 2, module 9.

In [14]:
modWanted = 9
rowWanted = 2
customname = '_Row_2_Module_09' # This is useful if we want to do various analysis.
trackerdict = demo.analysis1axis(trackerdict, modWanted=9, rowWanted = 2, customname=customname)

Linescan in process: 1axis_-5.0_Row_2_Module_09_Front
Linescan in process: 1axis_-5.0_Row_2_Module_09_Back
Saved: results\irr_1axis_-5.0_Row_2_Module_09.csv
Index: -5.0. Wm2Front: 737969.9259259258. Wm2Back: 103511.26148148149
Linescan in process: 1axis_0.0_Row_2_Module_09_Front
Linescan in process: 1axis_0.0_Row_2_Module_09_Back
Saved: results\irr_1axis_0.0_Row_2_Module_09.csv
Index: 0.0. Wm2Front: 4992.931111111112. Wm2Back: 609.6241185185185
Linescan in process: 1axis_5.0_Row_2_Module_09_Front
Linescan in process: 1axis_5.0_Row_2_Module_09_Back
Saved: results\irr_1axis_5.0_Row_2_Module_09.csv
Index: 5.0. Wm2Front: 760179.5777777777. Wm2Back: 103316.52851851853

Saving Cumulative results
Saved: cumulative_results__Row_2_Module_09.csv


Let's look at the results with more detail. The analysis1axis routine created individual result .csv files for each angle, as well as one cumulative result .csv where the irradiance is added by sensor.


In [15]:
results = load.read1Result('results\cumulative_results__Row_2_Module_09.csv')
results

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,22.61212,-1.004,2.46,2.37,a8.1.a0.Custom_Tracker_Module.6457,a8.1.a0.Custom_Tracker_Module.2310,1.500079e+06,217173.584333,0.144775
1,22.26212,-1.004,2.46,2.37,a8.1.a0.Custom_Tracker_Module.6457,a8.1.a0.Custom_Tracker_Module.2310,1.500218e+06,210772.970367,0.140495
2,21.91212,-1.004,2.46,2.37,a8.1.a0.Custom_Tracker_Module.6457,a8.1.a0.Custom_Tracker_Module.2310,1.500313e+06,206189.951700,0.137431
3,21.56212,-1.004,2.46,2.37,a8.1.a0.Custom_Tracker_Module.6457,a8.1.a0.Custom_Tracker_Module.2310,1.500407e+06,198467.735600,0.132276
4,21.21212,-1.004,2.46,2.37,a8.1.tube1.16,sky,1.568887e+06,186642.728300,0.118965
5,20.86212,-1.004,2.46,2.37,a8.1.a1.Custom_Tracker_Module.6457,a8.1.a1.Custom_Tracker_Module.2310,1.499679e+06,201038.626767,0.134054
6,20.51212,-1.004,2.46,2.37,a8.1.a1.Custom_Tracker_Module.6457,a8.1.a1.Custom_Tracker_Module.2310,1.496805e+06,209103.182300,0.139700
7,20.16212,-1.004,2.46,2.37,a8.1.a1.Custom_Tracker_Module.6457,a8.1.a1.Custom_Tracker_Module.2310,1.496917e+06,215010.665800,0.143636
8,19.81212,-1.004,2.46,2.37,a8.1.a1.Custom_Tracker_Module.6457,a8.1.a1.Custom_Tracker_Module.2310,1.497009e+06,222066.050233,0.148340


There are various things to notice:

I. The materials column has a specific format that will tell you if you are sampling the correct module:

                                a{ModWanted}.{rowWanted}.a{numPanel}.{moduletype}.material_key

* Since for this journal numPanels = 2, numPanel can either be 0 or 1, for the East-most and West-most module in the collector width.
* numPanel, ModWanted and RowWanted are indexed starting at 0 in the results.
* material_key is from the surface generated inside radiance. Usually it is 6457 for top surface of hte module and .2310 for the bottom one. 

II. Sensors sample always in the same direction. For this N-S aligned tracker, that is East-most to West. For this 2-up portrait tracker which is 3.5 meters, 20x7 rows and we are sampling module 9 on row 2, the East to West sampling goes from 22.6 m to 19.81 m = 2.79m. It is not exatly 3.5 because the sensors are spaced evenly through the collector width (CW): 


![Sensors spaced along collector width](../images_wiki/Journal2Pics/spaced_sensors.png)

III. When there is a ygap in the collector width (2-UP or more configuration), some of the sensors might end up sampling the torque tube, or the sky. You can ses that in the materials columns. This also happens if the number of sensors is quite high, the edges of the module might be sampled instead of the sensors. For this reason, before calculating bifacial gain these results must be cleaned. For more advanced simulations, make sure you clean each result csv file individually.  We provide some options on load.py but some are very use-specific, so you might have to develop your own cleaning tool (or let us know on issues!)

<div class="alert alert-warning">
Important: If you have torquetubes and y-gap values, make sure you clean your results.
</div>


<a id='step11'></a>

## 11. Clean Results

We have two options for cleaning results. The simples on is <b>load.cleanResults</b>, but there is also a deepClean for specific purposes.

cleanResults will find materials that should not have values and set them to NaN.

In [16]:
results_clean = load.cleanResult(results)
results_clean


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,22.61212,-1.004,2.46,2.37,a8.1.a0.Custom_Tracker_Module.6457,a8.1.a0.Custom_Tracker_Module.2310,1.500079e+06,217173.584333,0.144775
1,22.26212,-1.004,2.46,2.37,a8.1.a0.Custom_Tracker_Module.6457,a8.1.a0.Custom_Tracker_Module.2310,1.500218e+06,210772.970367,0.140495
2,21.91212,-1.004,2.46,2.37,a8.1.a0.Custom_Tracker_Module.6457,a8.1.a0.Custom_Tracker_Module.2310,1.500313e+06,206189.951700,0.137431
3,21.56212,-1.004,2.46,2.37,a8.1.a0.Custom_Tracker_Module.6457,a8.1.a0.Custom_Tracker_Module.2310,1.500407e+06,198467.735600,0.132276
4,21.21212,-1.004,2.46,2.37,a8.1.tube1.16,sky,NaN,NaN,0.118965
5,20.86212,-1.004,2.46,2.37,a8.1.a1.Custom_Tracker_Module.6457,a8.1.a1.Custom_Tracker_Module.2310,1.499679e+06,201038.626767,0.134054
6,20.51212,-1.004,2.46,2.37,a8.1.a1.Custom_Tracker_Module.6457,a8.1.a1.Custom_Tracker_Module.2310,1.496805e+06,209103.182300,0.139700
7,20.16212,-1.004,2.46,2.37,a8.1.a1.Custom_Tracker_Module.6457,a8.1.a1.Custom_Tracker_Module.2310,1.496917e+06,215010.665800,0.143636
8,19.81212,-1.004,2.46,2.37,a8.1.a1.Custom_Tracker_Module.6457,a8.1.a1.Custom_Tracker_Module.2310,1.497009e+06,222066.050233,0.148340


These are the total irradiance values over all the hours of the year that the module at each sampling point will receive. Dividing the back irradiance average by the front irradiance average will give us the bifacial gain for the year:

![Bifacial Gain in Irradiance Formula](../images_wiki/Journal1Pics/BGG_Formula.png)

Assuming that our module from Prism Solar has a bifaciality factor (rear to front performance) of 90%, our <u> bifacial gain </u> is of:

In [17]:
bifacialityfactor = 0.9
print('Annual bifacial ratio: %0.3f ' %( np.nanmean(results_clean.Wm2Back) * bifacialityfactor / np.nanmean(results_clean.Wm2Front)) )

Annual bifacial ratio: 0.126 


<a id='condensed'></a>

## CONDENSED VERSION
Everything we've done so far in super short condensed version:

In [18]:
albedo = 0.25
lat = 37.5
lon = -77.6
nMods = 20
nRows = 7
hub_height = 2.3
gcr = 0.33
moduletype = 'Custom Tracker Module'  # this must already exist since we are not calling makeModule in this CONDENSED example.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal2'
limit_angle = 60
angeldelta = 5
backtrack = True
gcr = gcr
modWanted = 9
rowWanted = 2
cumulativesky = True

import bifacial_radiance
demo = RadianceObj(path = testfolder) 
demo.setGround(albedo)
epwfile = demo.getEPW(lat,lon) 
metdata = demo.readEPW(epwfile)
demo.set1axis(limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, cumulativesky = cumulativesky)
demo.genCumSky1axis()
sceneDict = {'gcr': gcr,'height':hub_height, 'nMods': nMods, 'nRows': nRows}  # orientation deprecated on v.0.2.4.
demo.makeScene1axis(moduletype=moduletype,sceneDict = sceneDict)
demo.makeOct1axis()
demo.analysis1axis(modWanted = modWanted, rowWanted = rowWanted)


path = C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal2
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!


C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:449: RuntimeWarning: invalid value encountered in arccos
  wc = np.degrees(np.arccos(temp))
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:473: RuntimeWarning: invalid value encountered in arccos
  aoi = np.degrees(np.arccos(np.abs(np.sum(sun_vec*panel_norm, axis=0))))
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:568: RuntimeWarning: invalid value encountered in remainder
  surface_azimuth = surface_azimuth % 360
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:572: RuntimeWarning: invalid value encountered in arccos
  surface_tilt = 90 - np.degrees(np.arccos(dotproduct))


Saving file EPWs\1axis_-60.0.csv, # points: 364
Saving file EPWs\1axis_-55.0.csv, # points: 136
Saving file EPWs\1axis_-50.0.csv, # points: 169
Saving file EPWs\1axis_-45.0.csv, # points: 146
Saving file EPWs\1axis_-40.0.csv, # points: 173
Saving file EPWs\1axis_-35.0.csv, # points: 146
Saving file EPWs\1axis_-30.0.csv, # points: 157
Saving file EPWs\1axis_-25.0.csv, # points: 175
Saving file EPWs\1axis_-20.0.csv, # points: 110
Saving file EPWs\1axis_-15.0.csv, # points: 136
Saving file EPWs\1axis_-10.0.csv, # points: 362
Saving file EPWs\1axis_-5.0.csv, # points: 141
Saving file EPWs\1axis_0.0.csv, # points: 81
Saving file EPWs\1axis_5.0.csv, # points: 244
Saving file EPWs\1axis_10.0.csv, # points: 81
Saving file EPWs\1axis_15.0.csv, # points: 123
Saving file EPWs\1axis_20.0.csv, # points: 225
Saving file EPWs\1axis_25.0.csv, # points: 154
Saving file EPWs\1axis_30.0.csv, # points: 116
Saving file EPWs\1axis_35.0.csv, # points: 167
Saving file EPWs\1axis_40.0.csv, # points: 135
Saving

Linescan in process: 1axis_30.0_Back
Saved: results\irr_1axis_30.0.csv
Index: 30.0. Wm2Front: 50298.879629629635. Wm2Back: 5706.771296296296
Linescan in process: 1axis_35.0_Front
Linescan in process: 1axis_35.0_Back
Saved: results\irr_1axis_35.0.csv
Index: 35.0. Wm2Front: 74908.2585185185. Wm2Back: 8393.601
Linescan in process: 1axis_40.0_Front
Linescan in process: 1axis_40.0_Back
Saved: results\irr_1axis_40.0.csv
Index: 40.0. Wm2Front: 53669.156666666655. Wm2Back: 5906.931444444444
Linescan in process: 1axis_45.0_Front
Linescan in process: 1axis_45.0_Back
Saved: results\irr_1axis_45.0.csv
Index: 45.0. Wm2Front: 75872.53370370371. Wm2Back: 7519.38462962963
Linescan in process: 1axis_50.0_Front
Linescan in process: 1axis_50.0_Back
Saved: results\irr_1axis_50.0.csv
Index: 50.0. Wm2Front: 56625.33333333333. Wm2Back: 5394.450148148148
Linescan in process: 1axis_55.0_Front
Linescan in process: 1axis_55.0_Back
Saved: results\irr_1axis_55.0.csv
Index: 55.0. Wm2Front: 77579.87962962962. Wm2Bac

{-5.0: {'csvfile': 'EPWs\\1axis_-5.0.csv',
  'surf_azm': 90.0,
  'surf_tilt': 5.0,
  'datetime': Index(['1955-01-01 08:00:00', '1955-01-02 08:00:00', '1955-01-03 08:00:00',
         '1955-01-04 08:00:00', '1955-01-05 08:00:00', '1955-01-06 08:00:00',
         '1955-01-07 08:00:00', '1955-01-08 08:00:00', '1955-01-09 08:00:00',
         '1955-01-10 08:00:00',
         ...
         '1959-12-22 08:00:00', '1959-12-23 08:00:00', '1959-12-24 08:00:00',
         '1959-12-25 08:00:00', '1959-12-26 08:00:00', '1959-12-27 08:00:00',
         '1959-12-28 08:00:00', '1959-12-29 08:00:00', '1959-12-30 08:00:00',
         '1959-12-31 08:00:00'],
        dtype='object', length=141),
  'count': 141,
  'skyfile': 'skies\\1axis_-5.0.rad',
  'clearance_height': 2.156193024466364,
  'radfile': 'objects\\1axis-5.0_2.156_10.60_5.0_20x7_origin0,0.rad',
  'scene': <bifacial_radiance.main.SceneObj at 0x14d3983e4a8>,
  'octfile': '1axis_-5.0.oct',
  'AnalysisObj': <bifacial_radiance.main.AnalysisObj at 0x14d3a